# SK-1-Fundamentals : Introduction a Semantic Kernel

**Navigation** : [Index](README.md) | [02-Functions >>](02-SemanticKernel-Advanced.ipynb)

---

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Installer et configurer **Semantic Kernel** pour Python
2. Comprendre le role du **Kernel** comme orchestrateur central
3. Ajouter des **services LLM** (OpenAI, Azure, Hugging Face)
4. Charger et utiliser des **plugins** (prompts templates)
5. Creer des **fonctions semantiques inline**
6. Gerer un **chat conversationnel** avec historique

### Prerequis

- Python 3.10+
- Cle API OpenAI (ou Azure OpenAI)
- Fichier `.env` configure

### Duree estimee : 45 minutes

---

## Sommaire

| Section | Contenu | Concepts cles |
|---------|---------|---------------|
| 1 | Installation | pip install semantic-kernel |
| 2 | Configuration | .env, Kernel, Services |
| 3 | Premier Kernel | Initialisation, service LLM |
| 4 | Plugins | Chargement depuis fichiers |
| 5 | Fonctions inline | Prompts dynamiques |
| 6 | Chat | Historique, KernelArguments |

> **Qu'est-ce que Semantic Kernel ?** Un SDK Microsoft open-source pour integrer des LLMs dans vos applications. Il orchestre les appels aux modeles, gere les plugins, et permet de creer des agents intelligents.

In [1]:
# Installation de Semantic Kernel (si nécessaire)
%pip install semantic-kernel

# Vérification de la version installée
from semantic_kernel import __version__
print(f"Semantic Kernel version : {__version__}")


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Semantic Kernel version : 1.39.2


## 📚 Importation des bibliothèques

Avant de commencer, nous devons importer les modules Python nécessaires.

**Modules clés** :
- `semantic_kernel` : SDK principal de Semantic Kernel
- `dotenv` : Chargement des variables d'environnement depuis `.env`
- `OpenAIChatCompletion` : Connecteur pour l'API OpenAI
- `KernelArguments` : Passage de paramètres dynamiques aux fonctions

> **Note** : Assurez-vous d'avoir exécuté `pip install semantic-kernel python-dotenv` avant cette cellule.

In [2]:
import os
import json
from dotenv import load_dotenv
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import KernelArguments  # Correction de l'import


## **📁 2. Chargement des paramètres de configuration**
### 📌 **Lecture des paramètres depuis un fichier `.env` ou JSON**
Le fichier de configuration `.env` doit contenir les clés nécessaires pour accéder aux services OpenAI/Azure OpenAI.

💡 **Vérifiez que vous avez bien créé un fichier `.env`** dans le même dossier que ce notebook avec ces valeurs :

```plaintext
GLOBAL_LLM_SERVICE="OpenAI"
OPENAI_API_KEY="sk-..."
OPENAI_CHAT_MODEL_ID="gpt-5-mini"
```

👉 Nous allons maintenant **charger ces paramètres en Python** :

In [3]:
# Chargement du fichier .env
load_dotenv()

# Récupération des clés API et du modèle
llm_service = os.getenv("GLOBAL_LLM_SERVICE", "OpenAI")
api_key = os.getenv("OPENAI_API_KEY")
model_id = os.getenv("OPENAI_CHAT_MODEL_ID", "gpt-5-mini")

# Vérification
print(f"Service sélectionné : {llm_service}")
print(f"Modèle utilisé : {model_id}")


Service sélectionné : OpenAI
Modèle utilisé : gpt-5-mini


### Interprétation : Configuration chargée

Le fichier `.env` a été chargé avec succès et les paramètres ont été récupérés.

**Vérification des valeurs** :
- **Service** : OpenAI (pas Azure)
- **Modèle** : gpt-5-mini (version rapide et économique de GPT-5)
- **Clé API** : Présente dans l'environnement (non affichée pour des raisons de sécurité)

**Points importants** :

| Variable | Utilité |
|----------|---------|
| `GLOBAL_LLM_SERVICE` | Permet de basculer entre OpenAI/Azure/Hugging Face |
| `OPENAI_API_KEY` | Authentification auprès de l'API OpenAI |
| `OPENAI_CHAT_MODEL_ID` | Spécifie quel modèle utiliser (gpt-5-mini, gpt-4o, etc.) |

> **Sécurité** : Ne jamais commit le fichier `.env` dans Git ! Utilisez `.env.example` comme template et ajoutez `.env` dans `.gitignore`.

In [11]:
# Importer le Kernel depuis Semantic Kernel
from semantic_kernel import Kernel

# Créer une instance du Kernel
kernel = Kernel()
print("Kernel initialisé avec succès.")


Kernel initialisé avec succès.


### Interprétation : Initialisation du Kernel

Le **Kernel** est l'objet central de Semantic Kernel. Il a été créé avec succès.

**Rôle du Kernel** :
- **Orchestrateur** : Coordonne les services LLM, plugins et fonctions
- **Registry** : Maintient un registre de tous les composants disponibles
- **Executor** : Gère l'exécution des fonctions et la propagation des arguments

**État actuel** :
- Kernel créé mais **vide** (aucun service LLM ajouté)
- Prêt à recevoir des services via `kernel.add_service()`
- Prêt à charger des plugins via `kernel.add_plugin()`

> **Prochaine étape** : Nous devons maintenant ajouter un **service de Chat Completion** (OpenAI, Azure, etc.) pour permettre au Kernel d'invoquer des LLMs.

In [ ]:
import os
from dotenv import load_dotenv
from openai import AsyncOpenAI
import asyncio

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
# Récupère la valeur du .env et nettoie les espaces
base_url_from_env = os.getenv("OPENAI_BASE_URL", "").strip()

# Détermine explicitement l'URL finale
if base_url_from_env:
    final_base_url = base_url_from_env
    print(f"Utilisation de l'URL du .env : {final_base_url}")
else:
    final_base_url = "https://api.openai.com/v1" # <<< URL par défaut explicite
    print(f"Utilisation de l'URL par défaut : {final_base_url}")

model_id = os.getenv("OPENAI_CHAT_MODEL_ID", "gpt-5-mini")
print(f"Utilisation du modèle : {model_id}")
print(f"Clé API utilisée : {'Oui' if api_key else 'Non'}")

async def test_connection():
    try:
        if not api_key:
            print("ERREUR : Clé API non définie.")
            return

        # Utilise final_base_url déterminé ci-dessus
        client = AsyncOpenAI(api_key=api_key, base_url=final_base_url)
        response = await client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": "Say hello!"}],
            timeout=20
        )
        print("Connexion réussie !")
        print("Réponse :", response.choices[0].message.content)
    except Exception as e:
        print(f"ERREUR lors du test de connexion : {type(e).__name__} - {e}")
        import traceback
        traceback.print_exc()

# Exécuter le test
await test_connection()

## Configuration du service LLM

Nous devons connecter notre Kernel à un service de Chat Completion.  
Pour cet exemple, nous allons utiliser OpenAI. Si vous préférez Azure OpenAI, adaptez le code en conséquence (voir la documentation).

La configuration se fait via l'ajout d'un service au Kernel.  
Assurez-vous que votre fichier `.env` contient votre clé API.


In [12]:
# Importation du service OpenAI pour le Chat Completion
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

# Ajout du service "default" au Kernel
kernel.add_service(OpenAIChatCompletion(service_id="default"))
print("Service OpenAI ajouté au Kernel.")


Service OpenAI ajouté au Kernel.


### Interprétation : Service LLM ajouté

Le Kernel dispose maintenant d'un **service de Chat Completion** opérationnel.

**Configuration du service** :
- **Connecteur** : `OpenAIChatCompletion` (compatible avec l'API OpenAI standard)
- **Service ID** : "default" (identifiant utilisé pour référencer ce service)
- **Authentification** : Utilise automatiquement `OPENAI_API_KEY` du `.env`

**Cycle de vie du service** :

```
┌──────────────┐
│ kernel.add   │ ← Enregistrement du service
│  _service()  │
└──────┬───────┘
       │
       ▼
┌──────────────────────────────┐
│ Service "default" actif      │
│ - OpenAI API configurée      │
│ - Modèle gpt-5-mini par défaut│
└──────────────────────────────┘
       │
       ▼
┌──────────────────────────────┐
│ Prêt pour kernel.invoke()    │
└──────────────────────────────┘
```

> **Service multiples** : Vous pouvez ajouter plusieurs services avec des `service_id` différents (ex: "openai", "azure", "local"). Lors de l'invocation, spécifiez lequel utiliser via les `execution_settings`.

### Interpretation : Architecture du Kernel

Le Kernel est le **coeur** de Semantic Kernel. Il orchestre :

| Composant | Role | Exemple |
|-----------|------|---------|
| **Services** | Connexions aux LLMs | OpenAIChatCompletion, AzureChatCompletion |
| **Plugins** | Collections de fonctions | FunPlugin, WriterPlugin |
| **Fonctions** | Unites d'execution | Joke, Summarize, Chat |
| **Arguments** | Parametres dynamiques | KernelArguments(input=...) |

```
┌─────────────────────────────────────────┐
│                 Kernel                  │
│  ┌─────────────┐  ┌─────────────────┐  │
│  │  Services   │  │    Plugins      │  │
│  │  (LLMs)     │  │  (Functions)    │  │
│  └─────────────┘  └─────────────────┘  │
│           ↓              ↓              │
│       ┌─────────────────────────┐      │
│       │     invoke(func, args)  │      │
│       └─────────────────────────┘      │
└─────────────────────────────────────────┘
```

> **Service ID** : Chaque service a un identifiant unique. "default" est utilise si non specifie lors de l'invocation.

## Utiliser un Plugin de Prompt

Semantic Kernel permet de charger des **prompt plugins** stockés sur disque.  
Dans cet exemple, nous chargerons le plugin "FunPlugin" qui contient, par exemple, une fonction pour générer une blague.

Les fichiers du plugin (le prompt et sa configuration) sont stockés dans le répertoire `prompt_template_samples/`.  
Nous allons charger ce plugin et invoquer la fonction "Joke" pour générer une blague sur un sujet donné.


In [ ]:
# Chemin correct vers les plugins
plugins_directory = "./prompt_template_samples/"

# Vérifier si le dossier du plugin existe avant de charger
if os.path.exists(os.path.join(plugins_directory, "FunPlugin")):
    fun_plugin = kernel.add_plugin(parent_directory=plugins_directory, plugin_name="FunPlugin")
    joke_function = fun_plugin["Joke"]
    print("Plugin chargé avec succès.")
else:
    print("⚠️ Le plugin FunPlugin est introuvable. Vérifiez le chemin et assurez-vous qu'il est bien présent.")


# Invoquer la fonction pour générer une blague sur un thème donné
# Pour ce faire, nous utilisons des KernelArguments (ici, seul l'input est nécessaire)
from semantic_kernel.functions import KernelArguments

print(joke_function)

# Exemple : générer une blague sur "time travel to dinosaur age" avec un style "super silly"
# joke_response = await kernel.invoke(joke_function, KernelArguments(input="time travel to dinosaur age", style="super silly"))
joke_response = await kernel.invoke(joke_function, KernelArguments(input="time travel to dinosaur age", style="super silly"))
print("Blague générée :", joke_response)


Plugin chargé avec succès.
metadata=KernelFunctionMetadata(name='Joke', plugin_name='FunPlugin', description='Generate a funny joke', parameters=[KernelParameterMetadata(name='input', description='Joke subject', default_value='', type_='', is_required=True, type_object=None, schema_data={'type': 'object', 'description': 'Joke subject'}, include_in_function_choices=True), KernelParameterMetadata(name='style', description='Give a hint about the desired joke style', default_value='', type_='', is_required=True, type_object=None, schema_data={'type': 'object', 'description': 'Give a hint about the desired joke style'}, include_in_function_choices=True)], is_prompt=True, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='The completion result', default_value=None, type_='FunctionResult', is_required=True, type_object=None, schema_data=None, include_in_function_choices=True), additional_properties=None) invocation_duration_histogram=<opentelemetry.metr

### Interprétation : Utilisation d'un plugin

Le plugin **FunPlugin** a été chargé avec succès et la fonction `Joke` invoquée.

**Analyse du résultat** :
- Le prompt a injecté les paramètres `input="time travel to dinosaur age"` et `style="super silly"`
- Le modèle a généré une blague respectant les contraintes G-rated du template
- La réponse montre que le LLM a bien compris le style demandé ("super silly")

**Architecture du plugin sur disque** :

```
prompt_template_samples/
└── FunPlugin/
    └── Joke/
        ├── config.json       ← Métadonnées (description, paramètres)
        └── skprompt.txt      ← Template du prompt
```

**Avantages des plugins fichiers** :
- **Réutilisabilité** : Un même plugin peut être chargé dans plusieurs applications
- **Versioning** : Les prompts évoluent indépendamment du code
- **Non-code contributors** : Les experts métier peuvent éditer les prompts sans toucher au Python

> **Comparaison** : Les plugins fichiers sont idéaux pour des prompts stables et partagés, tandis que les fonctions inline (voir section suivante) sont meilleures pour des prompts générés dynamiquement.

## Définir une fonction sémantique en ligne

Outre l'utilisation de plugins stockés sur disque, il est possible de définir des fonctions sémantiques directement dans votre code Python.  
Cette approche est particulièrement utile pour :
- Générer dynamiquement des prompts en fonction du contexte
- Prototyper rapidement des idées sans créer de fichiers séparés

Dans cet exemple, nous allons créer une fonction qui résume un texte donné en quelques mots (TL;DR).


In [ ]:
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.prompt_template.input_variable import InputVariable

# Définition du prompt
tldr_prompt = """
{{$input}}

Donne-moi un résumé en 5 mots ou moins.
"""

# Configuration de l'exécution
execution_settings = OpenAIChatPromptExecutionSettings(
    service_id="default",
    ai_model_id=model_id,
)

# Configuration du prompt template
tldr_template_config = PromptTemplateConfig(
    template=tldr_prompt,
    name="tldr",
    template_format="semantic-kernel",
    input_variables=[InputVariable(name="input", description="Texte à résumer", is_required=True)],
    execution_settings=execution_settings,
)

# Ajout de la fonction au Kernel
tldr_function = kernel.add_function(function_name="tldrFunction", plugin_name="tldrPlugin", prompt_template_config=tldr_template_config)

# Exécution de la fonction
async def run_tldr():
    input_text = "Demo était une poétesse grecque ancienne connue pour un unique poème gravé sur le Colosse de Memnon."
    tldr_summary = await kernel.invoke(tldr_function, KernelArguments(input=input_text))
    print("Résumé (TL;DR) :", tldr_summary)

# Lancer la fonction
await run_tldr()

### Interprétation : Fonction TL;DR

Cette fonction démontre la **création dynamique de prompts** sans fichiers séparés.

**Analyse du résultat** :
- Le modèle a compressé une phrase complexe (Demo, poétesse grecque, Colosse de Memnon) en **5 mots maximum**
- Cette approche est idéale pour des tâches de synthèse rapide dans une application

**Configuration technique** :

| Paramètre | Valeur | Rôle |
|-----------|--------|------|
| `template_format` | "semantic-kernel" | Format des variables `{{$input}}` |
| `input_variables` | `[InputVariable(...)]` | Définit les paramètres attendus |
| `execution_settings` | `OpenAIChatPromptExecutionSettings` | Spécifie le modèle et service |

> **Avantage clé** : Cette méthode permet de générer des prompts **dynamiquement** en fonction du contexte applicatif, sans créer de fichiers `.json` et `.txt`.

## Chat interactif avec le Kernel

Semantic Kernel offre également la possibilité de créer des interactions de type chatbot.  
Nous allons configurer une fonction de chat qui utilise des **Kernel Arguments** pour conserver l'historique de la conversation.

L'objectif est de simuler une conversation où l'utilisateur envoie un message, le bot y répond, et l'historique est mis à jour à chaque échange.


In [ ]:
from semantic_kernel.contents import ChatHistory

# Initialiser l'historique
chat_history = ChatHistory()
chat_history.add_system_message("Vous êtes un chatbot utile et vous fournissez des recommandations de livres.")

# Définition du prompt de chat
chat_prompt = """
{{$history}}
User: {{$user_input}}
ChatBot:
"""

# Configuration de l'exécution
chat_exec_settings = OpenAIChatPromptExecutionSettings(
    service_id="default",
    ai_model_id=model_id,
)

# Configuration du chat template
chat_template_config = PromptTemplateConfig(
    template=chat_prompt,
    name="chat",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="user_input", description="Message de l'utilisateur", is_required=True),
        InputVariable(name="history", description="Historique de la conversation", is_required=True),
    ],
    execution_settings=chat_exec_settings,
)

# Ajout au Kernel
chat_function = kernel.add_function(function_name="chat", plugin_name="chatPlugin", prompt_template_config=chat_template_config)

# Fonction asynchrone pour le chat
async def chat(input_text: str):
    print(f"Utilisateur : {input_text}")
    response = await kernel.invoke(chat_function, KernelArguments(
        user_input=input_text, 
        history=str(chat_history),
        allow_dangerously_set_content=True
    ))
    print(f"ChatBot : {response}")
    chat_history.add_user_message(input_text)
    chat_history.add_assistant_message(str(response))

# Exécution des exemples
await chat("Salut, je cherche des suggestions de livres sur la philosophie antique.")
await chat("Peux-tu m'en recommander quelques-uns ?")

### Interprétation : Chat conversationnel

Cette cellule démontre l'utilisation de **ChatHistory** pour maintenir le contexte d'une conversation.

**Observations** :
- Le chatbot **se souvient** du contexte grâce à `chat_history`
- Chaque message utilisateur et réponse assistant est ajouté à l'historique
- Le prompt inclut `{{$history}}` pour injecter le contexte complet

**Mécanisme** :

```
Tour 1:
  [System] Instructions initiales
  [User] Question 1
  [Assistant] Réponse 1
  
Tour 2:
  [System] Instructions initiales  ← Contexte conservé
  [User] Question 1                ← Contexte conservé
  [Assistant] Réponse 1            ← Contexte conservé
  [User] Question 2                ← Nouvelle question
  [Assistant] Réponse 2
```

> **Limite importante** : L'historique complet est envoyé à chaque tour, ce qui peut dépasser les limites de tokens pour de longues conversations. En production, utilisez des techniques de **summarization** ou **sliding window**.

# Conclusion

## Resume des concepts

| Concept | Description | Code cle |
|---------|-------------|----------|
| **Kernel** | Orchestrateur central SK | `Kernel()` |
| **Service** | Connexion LLM | `kernel.add_service(OpenAIChatCompletion(...))` |
| **Plugin** | Collection de fonctions | `kernel.add_plugin(parent_directory=..., plugin_name=...)` |
| **Fonction semantique** | Prompt template execute | `kernel.add_function(prompt_template_config=...)` |
| **KernelArguments** | Parametres dynamiques | `KernelArguments(input=..., style=...)` |
| **ChatHistory** | Historique conversation | `ChatHistory()` + `add_user_message()` |

## Points cles a retenir

1. **Le Kernel est le coeur de SK** - Il orchestre services, plugins et fonctions
2. **Les plugins sont modulaires** - Chargeables depuis fichiers ou definis inline
3. **KernelArguments permet le passage dynamique** - Variables injectees dans les templates
4. **ChatHistory preserve le contexte** - Essentiel pour les conversations multi-tours

## Pour aller plus loin

| Notebook suivant | Contenu |
|-----------------|---------|
| [02-Functions](02-SemanticKernel-Advanced.ipynb) | Function Calling moderne, Memory, Groundedness |
| [03-Agents](03-SemanticKernel-Agents.ipynb) | ChatCompletionAgent, AgentGroupChat |

---

**Navigation** : [Index](README.md) | [02-Functions >>](02-SemanticKernel-Advanced.ipynb)